In [228]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import bs4
from time import sleep
import re
import operator
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager

import pika
import sys

import requests
import bs4
import json
import datetime

In [4]:
# %pip install selenium
# %pip install webdriver_manager
# %pip install pika

In [5]:
URL = 'https://www.steamspy.com/search.php'

In [213]:
browser = webdriver.Chrome(ChromeDriverManager().install())

games = ['Grand Theft Auto V', 'Portal 2', 'The Witcher 3: Wild Hunt', 'Tomb Raider (2013)', 'The Elder Scrolls V: Skyrim',
         'Left 4 Dead 2', 'Borderlands 2', 'Fallout 4', 'PAYDAY 2', 'Grand Theft Auto IV', 'DOOM (2016)','BioShock',
         'Half-Life 2', 'Red Dead Redemption 2', 'Limbo','Counter-Strike: Global Offensive', 'Life is Strange',
         'Team Fortress 2', 'BioShock Infinite']

# games = ['Grand Theft Auto V', 'Portal-2']
# games = ['Portal-2']

gameInfos = []

for game in games:
    browser.get(URL)
#     browser.implicitly_wait(1)
    search_field = browser.find_element_by_class_name('form-control')
    search_field.send_keys(game)
    search_field.submit()
    getInfo = browser.find_element_by_xpath('/html/body/div[3]/div[2]/div[1]/div[2]/div[1]/div[1]/div/p').text
    
    if getInfo.find("Price:") != -1:
        cleanData = getInfo[getInfo.find("Price:"):].split('\n')
        del cleanData[1]
        wat700 = "".join(cleanData).replace("Owners", ";Owners").replace(': ', "=")
        dictionary = dict(subString.split('=') for subString in wat700.split(";")) 
        
        dictionary["Game"] = game
        gameInfos.append(dictionary)

browser.close()
jsonData = json.dumps(gameInfos)
print(jsonData)


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\Andreas Heick Laptop\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[{"Price": "$29.99", "Owners": "20,000,000 .. 50,000,000", "Game": "Grand Theft Auto V"}, {"Price": "$39.99", "Owners": "10,000,000 .. 20,000,000", "Game": "The Witcher 3: Wild Hunt"}, {"Price": "$4.99", "Game": "Tomb Raider (2013)"}, {"Price": "$59.99", "Owners": "100,000 .. 200,000", "Game": "The Elder Scrolls V: Skyrim"}, {"Price": "$9.99", "Owners": "5,000,000 .. 10,000,000", "Game": "Left 4 Dead 2"}, {"Price": "$49.99", "Game": "Borderlands 2"}, {"Price": "$59.99", "Owners": "200,000 .. 500,000", "Game": "Fallout 4"}, {"Price": "$9.99", "Owners": "10,000,000 .. 20,000,000", "Game": "PAYDAY 2"}, {"Price": "$19.99", "Owners": "2,000,000 .. 5,000,000", "Game": "Grand Theft Auto IV"}, {"Price": "$4.99", "Game": "DOOM (2016)"}, {"Price": "$19.99", "Game": "BioShock"}, {"Price": "$59.99", "Owners": "2,000,000 .. 5,000,000", "Game": "Red Dead Redemption 2"}, {"Price": "$6.99", "Game": "Limbo"}, {"Price": "$31.96", "Game": "Life is Strange"}, {"Price": "$29.99", "Owners": "2,000,000 .. 5,

In [238]:
connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='gamerevWebscraper', exchange_type='fanout')

channel.queue_declare(queue='webscrapeData')
channel.queue_declare(queue='webscrapeData2')

channel.queue_bind(exchange='gamerevWebscraper', queue="webscrapeData")
channel.queue_bind(exchange='gamerevWebscraper', queue="webscrapeData2")

channel.basic_publish(exchange='gamerevWebscraper', routing_key='', body=jsonData)
print(" [x] Sent %r" % jsonData)
connection.close()

print("Webscraper Cron job has run at %s" % datetime.datetime.now())

 [x] Sent '[{"Price": "$29.99", "Owners": "20,000,000 .. 50,000,000", "Game": "Grand Theft Auto V"}, {"Price": "$39.99", "Owners": "10,000,000 .. 20,000,000", "Game": "The Witcher 3: Wild Hunt"}, {"Price": "$4.99", "Game": "Tomb Raider (2013)"}, {"Price": "$59.99", "Owners": "100,000 .. 200,000", "Game": "The Elder Scrolls V: Skyrim"}, {"Price": "$9.99", "Owners": "5,000,000 .. 10,000,000", "Game": "Left 4 Dead 2"}, {"Price": "$49.99", "Game": "Borderlands 2"}, {"Price": "$59.99", "Owners": "200,000 .. 500,000", "Game": "Fallout 4"}, {"Price": "$9.99", "Owners": "10,000,000 .. 20,000,000", "Game": "PAYDAY 2"}, {"Price": "$19.99", "Owners": "2,000,000 .. 5,000,000", "Game": "Grand Theft Auto IV"}, {"Price": "$4.99", "Game": "DOOM (2016)"}, {"Price": "$19.99", "Game": "BioShock"}, {"Price": "$59.99", "Owners": "2,000,000 .. 5,000,000", "Game": "Red Dead Redemption 2"}, {"Price": "$6.99", "Game": "Limbo"}, {"Price": "$31.96", "Game": "Life is Strange"}, {"Price": "$29.99", "Owners": "2,00